In [1]:
%pip install nbimporter
%pip install nbformat

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
# CODE
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from urllib.parse import urlparse
import re
import nbimporter
import sys
import pandas as pd
import numpy as np 

data = pd.read_csv("C:/Users/Danny Phong/Documents/PROJECT/WebVulnerabilities/data/malicious_phish.csv")
print(data.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651191 entries, 0 to 651190
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     651191 non-null  object
 1   type    651191 non-null  object
dtypes: object(2)
memory usage: 9.9+ MB
None


In [3]:
# String processing
def extract_features(url):
    parsed = urlparse(url)
    features = {}
    features["url_length"] = len(url)
    features["num_dots"] = url.count(".")
    features["num_hyphens"] = url.count("-")
    features["num_at"] = url.count("@")
    features["uses_https"] = int(parsed.scheme == "https")
    features["has_ip"] = int(bool(re.search(r'\d+\.\d+\.\d+\.\d+', parsed.netloc)))
    features["num_subdomains"] = len(parsed.netloc.split(".")) - 2  
    features["path_length"] = len(parsed.path)
    return features

# Áp dụng cho tất cả URL trong DataFrame
features_df = data["url"].apply(extract_features).apply(pd.Series)

In [4]:
# Reduce dimensionality and merge back into the original data
# Áp dụng PCA để giảm số chiều
pca = PCA(n_components=2)  # Chỉ lấy một thành phần
reduced_features = pca.fit_transform(features_df)

pca_df = pd.DataFrame(reduced_features, columns=["pca1", "pca2"])

# Dùng để làm cho chỉ số khớp nhau trước khi nối 
pca_df.reset_index(drop=True, inplace=True)
data.reset_index(drop=True, inplace=True)

# Nối vào data gốc
data = pd.concat([data, pca_df], axis=1)

print(data.head())

                                                 url        type        pca1  \
0                                   br-icloud.com.br    phishing  -50.658225   
1                mp3raid.com/music/krizz_kaliko.html      benign  -25.420248   
2                    bopsecrets.org/rexroth/cr/1.htm      benign  -30.740088   
3  http://www.garage-pirenne.be/index.php?option=...  defacement   11.911550   
4  http://adventure-nicaragua.net/index.php?optio...  defacement  144.818209   

         pca2  
0   -3.906399  
1    5.123978  
2    3.215432  
3  -40.076771  
4 -102.743698  


In [5]:
# Cleaning data

# #known_exploited_vulnerabilities.csv
# data = data.dropna()
# data = data.drop(["dateAdded","cveID", "vulnerabilityName", "shortDescription", "dueDate", "notes"], axis=1)
# data =data.drop_duplicates()

#malicious_phish.csv
print(data["type"].unique())  # Kiểm tra tất cả giá trị duy nhất trong cột 'type'
data.replace("nan", np.nan, inplace=True)
data = data.dropna()
data = data.drop_duplicates()

statusMap = {"benign": 0, "phishing": 1, "defacement": 1, "malware": 1}
data["type"] = data["type"].map(statusMap)
print(data["type"].isna().any())

y = data["type"].values.ravel()

x = pd.DataFrame(data, columns=["pca1", "pca2"])

print(data.head())

# #csic_database.csv
# data = data.dropna()
# data = data.drop(["Pragma", "Cache-Control", "Accept", "Accept-encoding", "Accept-charset", "language", "cookie", "content", "classification"], axis=1)
# data =data.drop_duplicates()

['phishing' 'benign' 'defacement' 'malware']
False
                                                 url  type        pca1  \
0                                   br-icloud.com.br     1  -50.658225   
1                mp3raid.com/music/krizz_kaliko.html     0  -25.420248   
2                    bopsecrets.org/rexroth/cr/1.htm     0  -30.740088   
3  http://www.garage-pirenne.be/index.php?option=...     1   11.911550   
4  http://adventure-nicaragua.net/index.php?optio...     1  144.818209   

         pca2  
0   -3.906399  
1    5.123978  
2    3.215432  
3  -40.076771  
4 -102.743698  
